In [657]:
import pandas as pd
import numpy as np # type: ignore
from pybaseball import *
import statsapi
import os

from pull_data import *
%load_ext autoreload
%autoreload 2
import requests

from pybaseball import cache
cache.enable()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [658]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

team_ids = [138, 140, 145, 110, 111, 144, 158, 143, 133, 141, 108, 114, 115, 121, 137, 146, 116, 118, 139, 117, 113, 112, 142, 136, 120, 109, 134, 135, 147, 119]

In [659]:
# Update roster player ids
reload = False
if reload:
    player_id_mapping = map_player_ids(team_ids)
    player_id_mapping.to_csv('player_id_mapping.csv', index=False)
else:
    player_id_mapping = pd.read_csv('player_id_mapping.csv')

In [660]:
player_id_mapping

,name,abbrev,clean_name,clean_abbrev,team,player_id
0,Alec Burleson,A. Burleson,Alec Burleson,A. Burleson,Cardinals,676475
1,Andre Pallante,A. Pallante,Andre Pallante,A. Pallante,Cardinals,669467
2,Brendan Donovan,B. Donovan,Brendan Donovan,B. Donovan,Cardinals,680977
3,Chris Roycroft,C. Roycroft,Chris Roycroft,C. Roycroft,Cardinals,688297
4,Erick Fedde,E. Fedde,Erick Fedde,E. Fedde,Cardinals,607200
...,...,...,...,...,...,...
772,Tanner Scott,T. Scott,Tanner Scott,T. Scott,Dodgers,656945
773,Teoscar Hernández,T. Hernández,Teoscar Hernandez,T. Hernandez,Dodgers,606192
774,Tommy Edman,T. Edman,Tommy Edman,T. Edman,Dodgers,669242
775,Will Smith,W. Smith,Will Smith,W. Smith,Dodgers,519293


In [661]:
from datetime import date

# Get today's date in YYYY-MM-DD format
check_yesterday = False
days_offset = 0
today = (date.today() + pd.DateOffset(days=days_offset)).strftime('%Y-%m-%d')
yesterday = (date.today() + pd.DateOffset(days=days_offset - 1)).strftime('%Y-%m-%d')
if check_yesterday:
    today = yesterday

# Get today's schedule
games = statsapi.schedule(start_date=today, end_date=today)

# Loop through games and extract probable pitchers
for game in games:
    home_team = game['home_name']
    away_team = game['away_name']
    home_pitcher = game.get('home_probable_pitcher', 'TBD')
    away_pitcher = game.get('away_probable_pitcher', 'TBD')
    
    print(f"{away_team} ({away_pitcher}) @ {home_team} ({home_pitcher})")

Cincinnati Reds (Wade Miley) @ Cleveland Guardians (Luis L. Ortiz)
Miami Marlins (Eury Pérez) @ Pittsburgh Pirates (Mike Burrows)
Chicago Cubs (Matthew Boyd) @ Philadelphia Phillies (Zack Wheeler)
Tampa Bay Rays (Shane Baz) @ Boston Red Sox (Brayan Bello)
Atlanta Braves (Chris Sale) @ Milwaukee Brewers (Aaron Civale)
Toronto Blue Jays (José Berríos) @ St. Louis Cardinals (Andre Pallante)
Athletics (Grant Holman) @ Los Angeles Angels (Yusei Kikuchi)
Los Angeles Dodgers (Dustin May) @ San Diego Padres (Nick Pivetta)
Seattle Mariners (Emerson Hancock) @ Arizona Diamondbacks (Merrill Kelly)


In [662]:
pitching_df, batting_df = get_rotowire_lineups()

In [663]:
import os
api_key = os.getenv("api_key")
print(api_key)

fa1d935c1d67073458a1f970bc434075


In [664]:
# Note that BBE = attempts = number of events
pitcher_barrels = statcast_pitcher_exitvelo_barrels(2025)

In [665]:
start_date = (date.today() + pd.DateOffset(days=-30)).strftime('%Y-%m-%d')
print("start date: ", start_date)
# start_date = '2025-05-01'
end_date = today
print("end date: ", end_date)
# end_date = '2025-06-03'
pitcher_ids = get_starting_ids(days_offset=days_offset)
pitcher_id_nos = [p[0] for p in pitcher_ids]
pitchers_arsenals = {}
pitcher_opponents = {}
pitcher_games = {}
for id, opposing_team, game_id in pitcher_ids:
    pitcher_data = statcast_pitcher(start_dt=start_date, end_dt=end_date,player_id=id)
    pitcher_opponents[id] = opposing_team
    pitcher_games[id] = game_id
    pitchers_arsenals[id] = {'R': get_pitch_arsenal(pitcher_data, 'R'), 'L': get_pitch_arsenal(pitcher_data, 'L')}

start date:  2025-05-10
end date:  2025-06-09
['Luis L. Ortiz', 'Wade Miley', 'Mike Burrows', 'Eury Pérez', 'Zack Wheeler', 'Matthew Boyd', 'Brayan Bello', 'Shane Baz', 'Aaron Civale', 'Chris Sale', 'Andre Pallante', 'José Berríos', 'Yusei Kikuchi', 'Grant Holman', 'Nick Pivetta', 'Dustin May', 'Merrill Kelly', 'Emerson Hancock']
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


In [666]:
# pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['attempts'] > 50]
pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['player_id'].isin(pitcher_id_nos)].copy()
pitcher_barrels_copy['barrel_percentage'] = pitcher_barrels_copy['player_id'].apply(get_barrel_percentage, start_date=start_date, end_date=end_date)
pitcher_barrels_copy['left_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['left_pct'])
pitcher_barrels_copy['right_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['right_pct'])
pitcher_barrels_copy['opponent'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_opponents[x])
pitcher_barrels_copy['game_id'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_games.get(x))
for index, row in pitcher_barrels_copy.iterrows():
    if row['barrel_percentage']['total_pct'] == 0:
        pitcher_barrels_copy.at[index, 'left_barrel_pct'] = row['brl_percent'] / 100
        pitcher_barrels_copy.at[index, 'right_barrel_pct'] = row['brl_percent'] / 100
    
pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'opponent']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,opponent
33,"Pallante, Andre",669467,777581,199,16,8.00,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.08,0.09,away
35,"Berríos, José",621244,777581,197,23,11.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.05,0.10,home
40,"Kelly, Merrill",518876,777577,196,20,10.20,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.03,0.09,away
51,"Kikuchi, Yusei",579328,777576,190,16,8.40,"{'left_barrels': 2, 'right_barrels': 4, 'total...",0.05,0.06,away
62,"Wheeler, Zack",554430,777578,179,12,6.70,"{'left_barrels': 2, 'right_barrels': 0, 'total...",0.04,0.00,away
64,"Baz, Shane",669358,777584,177,22,12.40,"{'left_barrels': 2, 'right_barrels': 9, 'total...",0.05,0.14,home
68,"Sale, Chris",519242,777575,175,16,9.10,"{'left_barrels': 2, 'right_barrels': 4, 'total...",0.10,0.04,home
71,"Boyd, Matthew",571510,777578,173,12,6.90,"{'left_barrels': 1, 'right_barrels': 5, 'total...",0.04,0.06,home
77,"May, Dustin",669160,777579,170,15,8.80,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.08,0.08,home
97,"Pivetta, Nick",601713,777579,156,18,11.50,"{'left_barrels': 4, 'right_barrels': 4, 'total...",0.08,0.07,away


In [667]:
filtered_pitcher_barrels_copy = pitcher_barrels_copy[(pitcher_barrels_copy['left_barrel_pct'] > 0.07) | (pitcher_barrels_copy['right_barrel_pct'] > 0.07)].copy()
filtered_pitcher_barrels_copy['sides'] = filtered_pitcher_barrels_copy.apply(lambda row: get_sides(row['left_barrel_pct'], row['right_barrel_pct']),axis=1)
filtered_pitcher_barrels_copy['arsenal'] = filtered_pitcher_barrels_copy.apply(lambda row: pitchers_arsenals[row['player_id']], axis=1)
filtered_pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent']]

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent
33,"Pallante, Andre",669467,777581,199,16,8.00,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.08,0.09,"[L, R]","{'R': {'FF': 29.842931937172775, 'SL': 29.8429...",away
35,"Berríos, José",621244,777581,197,23,11.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.05,0.10,[R],"{'R': {'SI': 39.53488372093023, 'SV': 34.30232...",home
40,"Kelly, Merrill",518876,777577,196,20,10.20,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.03,0.09,[R],"{'R': {'FC': 26.4, 'CH': 21.2, 'SI': 19.2}, 'L...",away
64,"Baz, Shane",669358,777584,177,22,12.40,"{'left_barrels': 2, 'right_barrels': 9, 'total...",0.05,0.14,[R],"{'R': {'FF': 47.1042471042471, 'KC': 35.521235...",home
68,"Sale, Chris",519242,777575,175,16,9.10,"{'left_barrels': 2, 'right_barrels': 4, 'total...",0.10,0.04,[L],"{'R': {'SL': 50.753768844221106, 'FF': 46.7336...",home
77,"May, Dustin",669160,777579,170,15,8.80,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.08,0.08,"[L, R]","{'R': {'ST': 46.308724832214764, 'SI': 42.9530...",home
97,"Pivetta, Nick",601713,777579,156,18,11.50,"{'left_barrels': 4, 'right_barrels': 4, 'total...",0.08,0.07,[L],"{'R': {'FF': 42.016806722689076, 'ST': 34.8739...",away
110,"Hancock, Emerson",676106,777577,148,11,7.40,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.09,0.04,[L],"{'R': {'SI': 33.529411764705884, 'FF': 25.8823...",home
130,"Bello, Brayan",678394,777584,127,11,8.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.06,0.10,[R],"{'R': {'SI': 50.22831050228311, 'ST': 33.78995...",away
280,"Holman, Grant",680880,777576,62,7,11.30,"{'left_barrels': 3, 'right_barrels': 6, 'total...",0.17,0.17,"[L, R]","{'R': {'FF': 46.3768115942029, 'FS': 31.884057...",home


Next step is
- For each batter in a lineup that a pitcher is facing, we want to get their barrel % against the pitcher's arsenal (the pitchers > 18%)
- Calculate the weighted pull air percentage (pull % * air %)
- Also eventually can add blast %, iso

In [668]:
pitcher_df = filtered_pitcher_barrels_copy.copy()
pitcher_df['hitter_team'] = pitcher_df.apply(lambda row: get_team(row['game_id'], row['opponent']), axis=1)
pitcher_df['opposing_lineup'] = pitcher_df.apply(lambda row: get_opposing_lineup(row['game_id'], row['opponent']), axis=1)
pitcher_df['opposing_lineup_names'] = pitcher_df.apply(lambda row: get_specific_lineup(row['hitter_team'], batting_df), axis=1)
pitcher_df['rotowire_ids'] = pitcher_df.apply(lambda row: get_player_ids(row['opposing_lineup_names'], player_id_mapping, row['hitter_team']), axis=1)
pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'opposing_lineup_names','hitter_team','rotowire_ids']]

Error fetching lineup for game 777577: list index out of range


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup,opposing_lineup_names,hitter_team,rotowire_ids
33,"Pallante, Andre",669467,777581,199,16,8.00,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.08,0.09,"[L, R]","{'R': {'FF': 29.842931937172775, 'SL': 29.8429...",away,"[666182, 665489, 680718, 672386, 543807, 66592...","[Bo Bichette, V. Guerrero, A. Barger, A. Kirk,...",Blue Jays,"[666182, 665489, 680718, 672386, 543807, 66592..."
35,"Berríos, José",621244,777581,197,23,11.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.05,0.10,[R],"{'R': {'SI': 39.53488372093023, 'SV': 34.30232...",home,"[663457, 691026, 680977, 575929, 671056, 57144...","[L. Nootbaar, Masyn Winn, B. Donovan, W. Contr...",Cardinals,"[663457, 691026, 680977, 575929, 671056, 57144..."
40,"Kelly, Merrill",518876,777577,196,20,10.20,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.03,0.09,[R],"{'R': {'FC': 26.4, 'CH': 21.2, 'SI': 19.2}, 'L...",away,None,"[J. Crawford, J. Polanco, J. Rodriguez, Cal Ra...",Mariners,"[641487, 593871, 677594, 663728, 668227, 64213..."
64,"Baz, Shane",669358,777584,177,22,12.40,"{'left_barrels': 2, 'right_barrels': 9, 'total...",0.05,0.14,[R],"{'R': {'FF': 47.1042471042471, 'KC': 35.521235...",home,"[680776, 646240, 647351, 596115, 701350, 69222...","[Jarren Duran, R. Devers, Abraham Toro, Trevor...",Red Sox,"[680776, 646240, 647351, 596115, 121697, 69222..."
68,"Sale, Chris",519242,777575,175,16,9.10,"{'left_barrels': 2, 'right_barrels': 4, 'total...",0.10,0.04,[L],"{'R': {'SL': 50.753768844221106, 'FF': 46.7336...",home,"[694192, 661388, 592885, 656555, 663662, 68621...","[J. Chourio, W. Contreras, C. Yelich, Rhys Hos...",Brewers,"[694192, 661388, 592885, 656555, 663662, 68621..."
77,"May, Dustin",669160,777579,170,15,8.80,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.08,0.08,"[L, R]","{'R': {'ST': 46.308724832214764, 'SI': 42.9530...",home,"[665487, 650333, 592518, 701538, 657757, 59342...","[F. Tatis, Luis Arraez, M. Machado, J. Merrill...",Padres,"[665487, 650333, 592518, 701538, 657757, 59342..."
97,"Pivetta, Nick",601713,777579,156,18,11.50,"{'left_barrels': 4, 'right_barrels': 4, 'total...",0.08,0.07,[L],"{'R': {'FF': 42.016806722689076, 'ST': 34.8739...",away,"[660271, 605141, 518692, 606192, 669257, 57197...","[S. Ohtani, Mookie Betts, F. Freeman, T. Herna...",Dodgers,"[660271, 605141, 518692, 606192, 519293, 57197..."
110,"Hancock, Emerson",676106,777577,148,11,7.40,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.09,0.04,[L],"{'R': {'SI': 33.529411764705884, 'FF': 25.8823...",home,"[682998, 606466, 672695, 647304, 553993, 66697...",[],D-backs,[]
130,"Bello, Brayan",678394,777584,127,11,8.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.06,0.10,[R],"{'R': {'SI': 50.22831050228311, 'ST': 33.78995...",away,"[666139, 664040, 650490, 666018, 691406, 66396...","[Josh Lowe, Brandon Lowe, Yandy Diaz, J. Arand...",Rays,"[666139, 664040, 650490, 666018, 691406, 66396..."
280,"Holman, Grant",680880,777576,62,7,11.30,"{'left_barrels': 3, 'right_barrels': 6, 'total...",0.17,0.17,"[L, R]","{'R': {'FF': 46.3768115942029, 'FS': 31.884057...",home,"[687263, 694384, 545361, 621493, 518595, 62103...","[Zach Neto, N. Schanuel, Mike Trout, Taylor Wa...",Angels,"[687263, 694384, 545361, 621493, 518595, 62103..."


In [669]:
hitter_barrels = statcast_batter_exitvelo_barrels(2025)

In [670]:
def get_hitter_barrels(hitter_barrels, hitter_ids, start_date, end_date, pitch_mix, sides):
    if hitter_ids is None or len(hitter_ids) == 0:
        return {}
    hitter_barrels_copy = hitter_barrels[hitter_barrels['player_id'].isin(hitter_ids)].copy()
    hitter_barrels_copy['barrel_data'] = hitter_barrels_copy['player_id'].apply(hitter_barrel_percentage, start_date=start_date, end_date=end_date, pitch_mix=pitch_mix, sides=sides)
    hitter_barrels_copy['barrel_percentage'] = hitter_barrels_copy.apply(lambda row: get_barrel_pcts(row['barrel_data']), axis=1)
    hitter_barrels_copy['barrel_qual'] = hitter_barrels_copy['barrel_percentage'].apply(lambda x: check_barrel_pcts(x))
    # print(hitter_barrels_copy['barrel_qual'])
    filtered = hitter_barrels_copy[hitter_barrels_copy['barrel_qual'].apply(lambda x: x != [0,0,0,0,0,0,0])].copy()
    # print(filtered)
    if len(filtered) > 3:
        # Create a new column with just the first element
        filtered['barrel_qual_first'] = filtered['barrel_qual'].apply(lambda x: x[0])
        # Sort by that column and get top 3
        top_3 = filtered.nlargest(3, 'barrel_qual_first')
        # Drop the temporary column if you don't need it
        top_3 = top_3.drop('barrel_qual_first', axis=1)
        return dict(zip(zip(top_3['last_name, first_name'], top_3['player_id']), top_3['barrel_qual']))
    else:
        return dict(zip(zip(filtered['last_name, first_name'], filtered['player_id']), filtered['barrel_qual']))

In [671]:
hitter_pitcher_df = pitcher_df.copy()
hitter_pitcher_df['hitter_barrels'] = hitter_pitcher_df.apply(lambda row: get_hitter_barrels(hitter_barrels, row['rotowire_ids'], start_date, end_date, row['arsenal'], row['sides']), axis=1)
hitter_pitcher_df['num_hitters'] = hitter_pitcher_df['hitter_barrels'].apply(lambda x: len(x))
hitter_pitcher_df['hitter_team'] = hitter_pitcher_df.apply(lambda row: get_team(row['game_id'], row['opponent']), axis=1)
hitter_pitcher_df['start_time'] = hitter_pitcher_df['game_id'].apply(lambda x: get_start_time(x))
hitter_pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'hitter_barrels', 'num_hitters', 'hitter_team', 'start_time']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup,hitter_barrels,num_hitters,hitter_team,start_time
33,"Pallante, Andre",669467,777581,199,16,8.00,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.08,0.09,"[L, R]","{'R': {'FF': 29.842931937172775, 'SL': 29.8429...",away,"[666182, 665489, 680718, 672386, 543807, 66592...","{('Springer, George', 543807): [0.227272727272...",3,Blue Jays,6:45 pm.
35,"Berríos, José",621244,777581,197,23,11.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.05,0.10,[R],"{'R': {'SI': 39.53488372093023, 'SV': 34.30232...",home,"[663457, 691026, 680977, 575929, 671056, 57144...","{('Contreras, Willson', 575929): [0.1641791044...",1,Cardinals,6:45 pm.
40,"Kelly, Merrill",518876,777577,196,20,10.20,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.03,0.09,[R],"{'R': {'FC': 26.4, 'CH': 21.2, 'SI': 19.2}, 'L...",away,None,"{('Raleigh, Cal', 663728): [0.3684210526315789...",2,Mariners,6:40 pm.
64,"Baz, Shane",669358,777584,177,22,12.40,"{'left_barrels': 2, 'right_barrels': 9, 'total...",0.05,0.14,[R],"{'R': {'FF': 47.1042471042471, 'KC': 35.521235...",home,"[680776, 646240, 647351, 596115, 701350, 69222...","{('Rafaela, Ceddanne', 678882): [0.10810810810...",2,Red Sox,7:10 pm.
68,"Sale, Chris",519242,777575,175,16,9.10,"{'left_barrels': 2, 'right_barrels': 4, 'total...",0.10,0.04,[L],"{'R': {'SL': 50.753768844221106, 'FF': 46.7336...",home,"[694192, 661388, 592885, 656555, 663662, 68621...","{('Yelich, Christian', 592885): [0.10958904109...",1,Brewers,6:40 pm.
77,"May, Dustin",669160,777579,170,15,8.80,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.08,0.08,"[L, R]","{'R': {'ST': 46.308724832214764, 'SI': 42.9530...",home,"[665487, 650333, 592518, 701538, 657757, 59342...","{('Machado, Manny', 592518): [0.13953488372093...",2,Padres,6:40 pm.
97,"Pivetta, Nick",601713,777579,156,18,11.50,"{'left_barrels': 4, 'right_barrels': 4, 'total...",0.08,0.07,[L],"{'R': {'FF': 42.016806722689076, 'ST': 34.8739...",away,"[660271, 605141, 518692, 606192, 669257, 57197...","{('Ohtani, Shohei', 660271): [0.17073170731707...",2,Dodgers,6:40 pm.
110,"Hancock, Emerson",676106,777577,148,11,7.40,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.09,0.04,[L],"{'R': {'SI': 33.529411764705884, 'FF': 25.8823...",home,"[682998, 606466, 672695, 647304, 553993, 66697...",{},0,D-backs,6:40 pm.
130,"Bello, Brayan",678394,777584,127,11,8.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.06,0.10,[R],"{'R': {'SI': 50.22831050228311, 'ST': 33.78995...",away,"[666139, 664040, 650490, 666018, 691406, 66396...","{('Caminero, Junior', 691406): [0.164556962025...",2,Rays,7:10 pm.
280,"Holman, Grant",680880,777576,62,7,11.30,"{'left_barrels': 3, 'right_barrels': 6, 'total...",0.17,0.17,"[L, R]","{'R': {'FF': 46.3768115942029, 'FS': 31.884057...",home,"[687263, 694384, 545361, 621493, 518595, 62103...","{('Ward, Taylor', 621493): [0.1891891891891892...",3,Angels,6:38 pm.


In [672]:
# Barrel percent, blast percent
all_keys = {}
teams = []
names = []
barrels = []
barrels_pa = []
blasts = []
pull_air = []
iso = []
total_bbe = []
hitter_side = []
new_batters_ids = []
new_batters_names = []
for index, row in hitter_pitcher_df.iterrows():
    for key in row['hitter_barrels']:
        all_keys[(row['hitter_team'], key)] = [f"{float(val):.3f}" for val in row['hitter_barrels'][key] if val not in ['L','R']]
        teams.append(row['hitter_team'])
        names.append(key)
        barrels.append(row['hitter_barrels'][key][0])
        barrels_pa.append(row['hitter_barrels'][key][1])
        blasts.append(row['hitter_barrels'][key][2])
        pull_air.append(row['hitter_barrels'][key][3])
        iso.append(row['hitter_barrels'][key][4])
        total_bbe.append(row['hitter_barrels'][key][6])
        hitter_side.append(row['hitter_barrels'][key][5])
        new_batters_ids.append(key[1])
        new_batters_names.append(key[0])
sorted_batters = all_keys.items()
sorted_batters = sorted(sorted_batters, key=lambda x: x[1], reverse=True)
# for batter in sorted_batters:
#     print(batter)

chosen_hitters = pd.DataFrame({
    'team': teams,
    'name': names,
    'barrel_percent_BBE': barrels,
    'barrel_percent_PA': barrels_pa,
    'blast_percent_BBE': blasts,
    'pull_air_percent_BBE': pull_air,
    'iso_AB': iso,
    'total_bbe': total_bbe,
    'hitter_side': hitter_side
})
chosen_hitters = chosen_hitters.sort_values(by='barrel_percent_BBE', ascending=False)
chosen_hitters

,team,name,barrel_percent_BBE,barrel_percent_PA,blast_percent_BBE,pull_air_percent_BBE,iso_AB,total_bbe,hitter_side
4,Mariners,"(Raleigh, Cal, 663728)",0.37,0.21,0.53,0.53,0.53,19,R
0,Blue Jays,"(Springer, George, 543807)",0.23,0.15,0.35,0.17,0.28,66,R
15,Angels,"(Ward, Taylor, 621493)",0.19,0.12,0.27,0.22,0.38,74,R
12,Dodgers,"(Muncy, Max, 571970)",0.18,0.12,0.33,0.24,0.32,66,L
11,Dodgers,"(Ohtani, Shohei, 660271)",0.17,0.12,0.46,0.18,0.32,82,L
16,Angels,"(Adell, Jo, 666176)",0.17,0.11,0.41,0.20,0.34,54,R
13,Rays,"(Caminero, Junior, 691406)",0.16,0.13,0.38,0.25,0.34,79,R
3,Cardinals,"(Contreras, Willson, 575929)",0.16,0.11,0.37,0.22,0.16,67,R
17,Angels,"(Neto, Zach, 687263)",0.16,0.11,0.33,0.19,0.23,80,R
10,Padres,"(Sheets, Gavin, 657757)",0.16,0.11,0.36,0.23,0.24,77,L


To do:
- Add ISO

In [673]:
# hoskins = statcast_batter(start_dt='2025-05-01', end_dt='2025-06-03', player_id=663728)
# abs = hoskins[(hoskins['events'].notna()) & (~hoskins['events'].isin(['walk','hit_by_pitch','sac_fly','fielder_error']))]
# mapping = {'single': 1, 'double': 2, 'triple': 3, 'home_run': 4}
# abs['bases'] = abs['events'].map(lambda x: mapping.get(x, 0))
# abs_total = len(abs[abs['bases'] != 0])
# abs_avg = (abs['bases'].sum() - abs_total) / abs.shape[0]
# abs_avg

In [674]:
chosen_hitters['player_id'] = chosen_hitters['name'].apply(lambda x: x[-1])
chosen_hitters['yesterday_outcomes'] = chosen_hitters.apply(lambda row: get_hitter_outcomes(row['player_id'], yesterday, yesterday, row['hitter_side']), axis=1)
chosen_hitters


Gathering Player Data
Gathering Player Data
Springer, George
Gathering Player Data
Ward, Taylor
Gathering Player Data
Muncy, Max
Gathering Player Data
Ohtani, Shohei
Gathering Player Data
Adell, Jo
Gathering Player Data
Caminero, Junior
Gathering Player Data
Contreras, Willson
Gathering Player Data
Neto, Zach
Gathering Player Data
Sheets, Gavin
Gathering Player Data
Barger, Addison
Gathering Player Data
Bichette, Bo
Gathering Player Data
Machado, Manny
Gathering Player Data
Arozarena, Randy
Gathering Player Data
Story, Trevor
Gathering Player Data
Gathering Player Data
Yelich, Christian
Gathering Player Data
Rafaela, Ceddanne


,team,name,barrel_percent_BBE,barrel_percent_PA,blast_percent_BBE,pull_air_percent_BBE,iso_AB,total_bbe,hitter_side,player_id,yesterday_outcomes
4,Mariners,"(Raleigh, Cal, 663728)",0.37,0.21,0.53,0.53,0.53,19,R,663728,0
0,Blue Jays,"(Springer, George, 543807)",0.23,0.15,0.35,0.17,0.28,66,R,543807,2
15,Angels,"(Ward, Taylor, 621493)",0.19,0.12,0.27,0.22,0.38,74,R,621493,4
12,Dodgers,"(Muncy, Max, 571970)",0.18,0.12,0.33,0.24,0.32,66,L,571970,2
11,Dodgers,"(Ohtani, Shohei, 660271)",0.17,0.12,0.46,0.18,0.32,82,L,660271,2
16,Angels,"(Adell, Jo, 666176)",0.17,0.11,0.41,0.20,0.34,54,R,666176,0
13,Rays,"(Caminero, Junior, 691406)",0.16,0.13,0.38,0.25,0.34,79,R,691406,1
3,Cardinals,"(Contreras, Willson, 575929)",0.16,0.11,0.37,0.22,0.16,67,R,575929,3
17,Angels,"(Neto, Zach, 687263)",0.16,0.11,0.33,0.19,0.23,80,R,687263,0
10,Padres,"(Sheets, Gavin, 657757)",0.16,0.11,0.36,0.23,0.24,77,L,657757,0


In [675]:
len(chosen_hitters[chosen_hitters['yesterday_outcomes'] > 1.5])

8

In [676]:
len(chosen_hitters)

18

In [677]:
len(chosen_hitters[chosen_hitters['yesterday_outcomes'] > 0])

10

In [678]:
new_batters_names

['Springer, George',
 'Barger, Addison',
 'Bichette, Bo',
 'Contreras, Willson',
 'Raleigh, Cal',
 'Arozarena, Randy',
 'Rafaela, Ceddanne',
 'Story, Trevor',
 'Yelich, Christian',
 'Machado, Manny',
 'Sheets, Gavin',
 'Ohtani, Shohei',
 'Muncy, Max',
 'Caminero, Junior',
 'Jansen, Danny',
 'Ward, Taylor',
 'Adell, Jo',
 'Neto, Zach']

In [679]:
new_batters_ids

[543807,
 680718,
 666182,
 575929,
 663728,
 668227,
 678882,
 596115,
 592885,
 592518,
 657757,
 660271,
 571970,
 691406,
 643376,
 621493,
 666176,
 687263]

In [ ]:

# add_players_to_history = True
# all_history = pd.read_csv('all_history.csv')
# if add_players_to_history:
#     all_history[today] = new_batters_ids
#     all_history.to_csv('all_history.csv', index=False)


ValueError: Length of values (18) does not match length of index (32)